In [1]:
%pip install sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached transformers-4.29.1-py3-none-any.whl (7.1 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached torch-2.0.1-cp311-cp311-win_amd64.whl (172.3 MB)
  Using cached torchvision-0.15.2-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl (14.8 MB)
  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl (8.3 MB)
  Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl (42.2 MB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached regex-2023.5

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [4]:
sentences = [
    "it caught him off guard that space smelled of seared steak",
    "she could not decide between painting her teeth or brushing her nails",
    "he thought there'd be sufficient time is he hid his watch",
    "the bees decided to have a mutiny against their queen",
    "the sign said there was road work ahead so she decided to speed up",
    "on a scale of one to ten, what's your favorite flavor of color?",
    "flying stinging insects rebelled in opposition to the matriarch"
]

In [5]:
embeddings = model.encode(sentences)
embeddings.shape

(7, 768)

In [8]:
from sentence_transformers.util import cos_sim

scores = cos_sim(embeddings[-1], embeddings[:-1])

print(sentences[-1])
for i, score in enumerate(scores[0]):
    print(f"{round(score.item(), 4)} | {sentences[i]}")

flying stinging insects rebelled in opposition to the matriarch
0.1232 | it caught him off guard that space smelled of seared steak
0.1967 | she could not decide between painting her teeth or brushing her nails
0.0523 | he thought there'd be sufficient time is he hid his watch
0.6084 | the bees decided to have a mutiny against their queen
0.1011 | the sign said there was road work ahead so she decided to speed up
-0.0492 | on a scale of one to ten, what's your favorite flavor of color?
